In [1]:
import os
if os.name=='nt':
    mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
    os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sys import getsizeof
import time
import gc

%matplotlib inline

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pickle

def save_pickle(x, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_pickle(filename):
    with open(filename, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [3]:
x_train_time0 = read_pickle('x_train_date_feats_0.pickle')
x_test_time = read_pickle('x_test_date_feats_0.pickle')

In [4]:
response = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=[0, 969])

y_train0 = response.values.ravel()

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
x_train_numeric0 = read_pickle('x_train_numeric_feat_0.pickle')
x_test_numeric = read_pickle('x_test_numeric_feat_0.pickle')

In [6]:
x_train0 =np.concatenate((x_train_numeric0, x_train_time0), axis=1)
x_test = np.concatenate((x_test_numeric, x_test_time), axis=1)
del x_train_numeric0, x_train_time0, x_test_numeric, x_test_time, response
gc.collect()

7

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_train0, y_train0,
                                                  test_size=0.2, 
                                                  random_state=0)

In [8]:
n_estimators = 400
clf = XGBClassifier(max_depth=14, n_estimators=n_estimators, 
                    base_score=0.0058, n_jobs=8, colsample_bytree=0.6,
                   min_child_weight=5, subsample=0.9,  
                    reg_lambda=4, silent=False, learning_rate=0.03)

clf.fit(x_train, y_train, eval_set=[(x_val, y_val)], 
        eval_metric='auc', early_stopping_rounds=10)

[0]	validation_0-auc:0.896777
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.898892
[2]	validation_0-auc:0.904331
[3]	validation_0-auc:0.90491
[4]	validation_0-auc:0.907307
[5]	validation_0-auc:0.908691
[6]	validation_0-auc:0.908826
[7]	validation_0-auc:0.910099
[8]	validation_0-auc:0.9116
[9]	validation_0-auc:0.912907
[10]	validation_0-auc:0.912614
[11]	validation_0-auc:0.912364
[12]	validation_0-auc:0.912635
[13]	validation_0-auc:0.912727
[14]	validation_0-auc:0.912597
[15]	validation_0-auc:0.912211
[16]	validation_0-auc:0.911989
[17]	validation_0-auc:0.912203
[18]	validation_0-auc:0.912282
[19]	validation_0-auc:0.912147
Stopping. Best iteration:
[9]	validation_0-auc:0.912907



XGBClassifier(base_score=0.0058, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=14, min_child_weight=5, missing=None, n_estimators=400,
       n_jobs=8, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=4, scale_pos_weight=1, seed=None,
       silent=False, subsample=0.9)

In [9]:
if clf.best_ntree_limit:
    n_estimators = int(clf.best_ntree_limit * 1.2)

clf.n_estimators = 30#n_estimators

clf.fit(x_train0, y_train0, 
        eval_set=[(x_train0, y_train0)], 
        eval_metric='auc')

preds = clf.predict_proba(x_train0)
preds = preds[:, 1]

thresholds = np.linspace(0.01, 0.99, 50)
mcc = np.array([matthews_corrcoef(y_train0, preds>thr) for thr in thresholds])
plt.plot(thresholds, mcc)
best_threshold = thresholds[mcc.argmax()]
print(mcc.max())

preds = (clf.predict_proba(x_test)[:, 1]>best_threshold).astype(np.int8)
sub = pd.read_csv('data/sample_submission.csv.zip', index_col=0)
sub['Response'] = preds
sub.to_csv('submission_numeric_time.csv.gz', compression='gzip')